In [8]:
import json
import requests, time


def readkey(key):
    # Read from colab user configuration first. if cannot find, read from local file
    ret=None
    #from google.colab import userdata
    #ret=userdata.get(key)
    
    if not ret:
        with open('config.json', 'r') as file:
            ret=json.load(file)
            print(f"{ret}")
    return ret


In [ ]:
import json
import requests, time

# Configure telegram API
#from google.colab import userdata
#telegram_api_key=userdata.get("telegram_key")

# For local testing only
telegram_api_key=readkey("telegram_key")

url=f"https://api.telegram.org/bot{telegram_api_key}"

In [5]:
# Create a place holder for all user sessions
usersessions=dict()
exit_signal=False

def getlastmessages(rmsg):
    r1=rmsg.json()
    r1=r1["result"]
    print(f"message queue size={len(r1)}")
    if len(r1)==0:  
        return
    
    # For loop to initalize the usersessions when setfirstid=True or find last message id
    for item in r1:
        keys=item.keys()
        
        if 'edited_message' in keys: # when previous prompt is modified to be the new prompt
            itm=item['edited_message']
        elif 'message' in keys:
            itm=item['message']
        else:
           raise Exception("Unknown message key!")

        chat_id=str(itm['chat']['id']) # convert chat id to string for dict type key 
        msg=itm['text']
        msg_id=itm['message_id']

        if chat_id in usersessions: # for a user exists in the session list
            # find the last message id and set to user session lastmsgid attribute
            if msg_id>usersessions[chat_id]['lastmsgid']:
                orfirstid=usersessions[chat_id]['firstid'] #keep the original firstid at this moment for the usersession under this chat_id
                orstatus=usersessions[chat_id]['status']
                usersessions.update({chat_id:{'firstid':orfirstid, 'lastmsgid':msg_id, "msg":msg,'status':orstatus}})
                #print(f"update {chat_id} with {usersessions[chat_id]}")
        else: # for a user not in the session list
            usersessions.update({chat_id:{'firstid':0,'msg':"", 'lastmsgid':0, 'status':'active'}})
            requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'type [question] with model selection [Sealion or Gemin or DeepSeek or Stable] at latest or type [quit]'}")

def setModelResponse(chat_id, msg):
    msgr=str.lower(msg)
    msgr=msgr.split(" ")
    modelstr=msgr[-1]
    strlen=len(msg)
    if modelstr=='stable':
        msg=msg[0:strlen-len(modelstr)]
        '''
        insert STABLEDIFFUSION model call here
        '''
        print(f"stable response {msg}")
    else:
        if modelstr=='sealion':
            msg=msg[0:strlen-len(modelstr)]
            '''
            insert sealion model call  here
            '''
            print(f"sealion response {msg}")
        elif modelstr=='deepseek':
            msg=msg[0:strlen-len(modelstr)]
            '''
            insert deepseek model call here
            '''
            print(f"deepseek response {msg}")
        elif modelstr=='gemini':
            msg=msg[0:strlen-len(modelstr)]
            '''
            insert gemini model call here
            '''
            print(f"gemini response {msg}")
        else:
            msg=f"unknown model marker {modelstr}"
        requests.get(url+f"/sendMessage?chat_id={chat_id}&text={msg}")
        requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'type [question] with model selection [Sealion or Gemin or DeepSeek or Stable] at latest or type [quit]'}")

def handleNewMessage():
    # update firstid with lastmsgid      
    exitsignal=False
    nkeys=usersessions.keys()
    keys=list(nkeys)
    for chat_id in keys:
        msg_id = usersessions[chat_id]['lastmsgid']
        firstid= usersessions[chat_id]['firstid']
        msg=usersessions[chat_id]['msg']
        status=usersessions[chat_id]['status']
        if msg_id>firstid : # update firstid with msg_id
            usersessions[chat_id]={'firstid':msg_id,"msg":msg, 'lastmsgid':msg_id,'status':status}
            print(f"for {chat_id}, new masg_id={msg_id}, firstid={firstid}")
            if firstid==0:
                continue 
        else: # msg_id==firstid
            continue
        
        print(f"new message coming for chat_id ({chat_id}), lastmsgid= {msg_id}, firstid={firstid}, msg={msg}, status={status}")
        
        if status=='active':
            if str.lower(msg)=='quit':
                #usersessions.pop(chat_id, None) # remove usersession from the session list
                usersessions[chat_id]={'firstid':msg_id,"msg":'quit','lastmsgid':msg_id,'status':'inactive'}
                print(f"{chat_id}={usersessions[chat_id]}")
                requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'Bye! If you want to rejoin, please type mjbot'}")
            elif str.lower(msg)=='exitsignal':
                exitsignal=True
                break
            else:
                setModelResponse(chat_id, msg)
        else: # status==inactive
            if str.lower(msg)=='mjbot':
                requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'type [question] with model selection [Sealion or Gemin or DeepSeek or Stable] at latest or type [quit]'}")
                usersessions[chat_id]={'firstid':0,"msg":msg, 'lastmsgid':msg_id,'status':'active'}
    # ENd for Loop
    return exitsignal


In [ ]:

# Create a loop routine to simulate the chatbot
anum=0
while True:
    keys=usersessions.keys()
    cnum=0
    for key in keys:
        if usersessions[key]['status']=='active':
            cnum+=1
    
    if anum!=cnum:
        print(f"Number of active user sessions={cnum}")
        anum=cnum

    r1=requests.get(url+"/getUpdates")
    time.sleep(1)

    getlastmessages(r1)
    if handleNewMessage():
        for chat_id in keys:
            requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'You are forced to exit by bot master!'}")
        break

print("End")